In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

------------------

##  <ins>Задание 1.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project1_desc`.

### 1. Необходимо выбрать метрику и привести аргументацию.

Целевая метрика - расходы на задание (т.к. цель теста - снижение расходов на задание). Расходы на задание = (сумма всех расходов)/(кол-во заданий).
Контрольная метрика - среднее значение дохода с задания (контрольная метрика должна указать в случае чего на негативное влияние теста) = (сумма NVP)/(кол-во заданий)

In [ ]:
hist_telesales = pd.read_csv('hist_telesales.csv')
hist_telesales.head(5)

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,1,0,90,0,-90
1,1,0,0,5,0,-5
2,2,0,0,68,0,-68
3,3,1,0,22,0,-22
4,4,1,0,22,0,-22


### 2. Альтернатива в критерии.

Предположение (гипотеза) о том, что среднее число расходов на задание снизится после внедрения системы,
которая будет сама звонить человеку
и в случае успеха перенаправлять задание оператору.


### 3.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

Односторонняя альтернатива.
Размер выборки был выведен с использованием формулы из лекции. Размер теста был выбран равным размеру контроля.

In [ ]:
prob = 0.90
imp_crit = 0.05
uplift = 0.05
k = 0.5

average_cost = hist_telesales['Расходы'].mean()
std_cost = hist_telesales['Расходы'].std()
uplift_cost = average_cost * uplift

z_alpha = sp.stats.norm.ppf(1 - imp_crit)
z_beta = sp.stats.norm.ppf(prob)

selection_test = ((z_alpha - z_beta)**2) * (std_cost**2) / (k * uplift_cost**2)
selection_control = ((z_alpha - z_beta)**2) * (std_cost**2) / ((1 - k) * uplift_cost**2)

print('Размер выборки тест:', round(selection_test))
print('Размер выборки контроль:', round(selection_test))
print('Размер выборки всего:', round(selection_test) + round(selection_control))

Размер выборки тест: 347
Размер выборки контроль: 347
Размер выборки всего: 694


### 3.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project1_sample {размер выборки}` и вписать <ins>код выборки</ins>:
36682

In [ ]:
control = pd.read_csv('Контроль.csv')
test = pd.read_csv('Тест.csv')

control.head(10)

,ID,Флаг дозвона,Флаг продажи,Расходы,PV,NPV
0,0,0,0,2,0,-2
1,1,0,0,55,0,-55
2,2,1,0,19,0,-19
3,3,1,0,11,0,-11
4,4,1,0,8,0,-8
5,5,1,0,20,0,-20
6,6,1,0,12,0,-12
7,7,0,0,39,0,-39
8,8,1,0,7,0,-7
9,9,0,0,57,0,-57


In [ ]:
control = pd.read_csv('Контроль.csv')
text = pd.read_csv('Тест.csv')

imp_crit = 0.05
t, p_value = sp.stats.ttest_ind(test['Расходы'], control['Расходы'], alternative='less', equal_var=False)
print('p-value:', p_value)
if (p_value <= imp_crit):
  print("Новый алгоритм дал значимое улучшение")
  if (test['NPV'].mean() >= control['NPV'].mean()):
    print("Контрольная метрика не ухудшилась, новый алгоритм можно запускать")
  else:
    print("Контрольная метрика ухудшилась, новый алгоритм необходимо доработать")
else:
  print("Новый алгоритм не дал значимого улучшения")

p-value: 0.052310043206935226
Новый алгоритм не дал значимого улучшения


### 4. Принятие решения. Расписать подробно с аргументами.

In [ ]:
Почти значимые результаты

------------------

##  <ins>Задание 2.</ins>

### 0. Запосить у бота в Телеграме условие командой `/get_project2_desc`.

In [ ]:
hist_credit = pd.read_csv('hist_credit_card.csv')
hist_credit.head(10)

,ID,Возраст,Доход клиента,Вероятность банкротства,Флаг утилизации счёта,Расходы,PV КК,PV услуги,NPV
0,0,19,21620.835463,0.138061,0,102,0,0,-102
1,1,27,24897.990863,0.035508,1,409,11686,1754,13031
2,2,50,23989.526947,0.098793,0,16,0,0,-16
3,3,18,38442.409756,0.365661,1,788,13738,1578,14528
4,4,24,21291.521612,0.036909,1,1048,6594,2213,7759
5,5,19,22957.947147,0.021514,1,110,33316,1692,34898
6,6,27,22217.477344,0.319460,0,57,0,0,-57
7,7,46,29584.430233,0.324886,1,371,5859,2529,8017
8,8,27,21730.692059,0.254344,1,712,9981,3758,13027
9,9,41,34909.175512,0.017871,1,131,8794,5315,13978


### 1. Необходимо выбрать метрику и привести аргументацию.

Целевая метрика - доходность КК + услуги: (сумма всех NPV)/ (количество заданий)
Целевая метрика - показатель успешности компании.
В качестве контрольной метрики можем проверить изменение среднее значение вероятности банкротства на утилизацию счёт: если средняя вероятность банкротства упадёт, то можно полагать, что новый алгоритм не сказался негативно на кредитовании, иначе банк может не захотеть внедрять новый алгоритм из-за рисков.

### 2. Альтернатива в критерии.

Альтернатива - это гипотеза что средняя доходность по КК + услугам увеличится



### 3. Нужно выбрать параметр(-ы), влияющий(-ие) на целевую метрику. Привести аргументацию.

Доход клиента и вероятность банкротства влияют напрямую на целевую метрику. Более высокий доход значит что клиент с большей вероятностью будет утилизировать счёт, а так же что он с большей вероятностью приобретёт услугу и потратит на неё больше денег. Вероятность банкротства увеличивает шанс что мы потратим деньги на клиента, но он не сможет их вернуть, что негативно скажется на целевой метрике.
Возраст клиента скорее всего не влияет напрямую на целевую метрику, но он может иметь корелляцию с вероятностью банкротства и с доходом клиента.

### 4.1. Каков размер выборки? Привести аргументацию и написать как получилось то или иное число.

In [ ]:
find_probability = 0.85
imp_crit = 0.05
uplift = 0.03
k = 0.5

average_NPV = hist_credit['NPV'].mean()
std_NPV = hist_credit['NPV'].std()
uplift_NPV = average_NPV * uplift

z_alpha = sp.stats.norm.ppf(1 - imp_crit)
z_beta = sp.stats.norm.ppf(find_probability)

selection_test = ((z_alpha - z_beta)**2) * (std_NPV**2) / (k * uplift_NPV**2)
selection_control = ((z_alpha - z_beta)**2) * (std_NPV**2) / ((1 - k) * uplift_NPV**2)

print('Размер выборки тест:', round(selection_test))
print('Размер выборки контроль:', round(selection_test))
print('Размер выборки всего:', round(selection_test) + round(selection_control))

Размер выборки тест: 5315
Размер выборки контроль: 5315
Размер выборки всего: 10630


Односторонняя альтернатива.
Размер выборки был выведен с использованием формулы из лекции. Размер теста был выбран равным размеру контроля.

### 4.2. Необходимо запросить у бота в Телеграме выборку полученного размера командой `/get_project2_sample {размер выборки}` и вписать <ins>код выборки</ins>:
69945

In [ ]:
control = pd.read_csv('Контроль 2.csv')
test = pd.read_csv('Тест 2.csv')

columns_mapping = {
    'Возраст': 'age',
    'Доход клиента': 'income',
    'Вероятность банкротства': 'bankruptcy',
    'Флаг утилизации счёта': 'utilization',
    'Расходы': 'cost',
    'PV услуги': 'PV_service',
}

# rename columns
control = control.rename(columns=columns_mapping)
test = test.rename(columns=columns_mapping)

control.head(10)

,ID,age,income,bankruptcy,utilization,cost,PV КК,PV_service,NPV
0,0,53,20720.666140,0.022005,1,257,7846,1340,8929
1,1,70,23235.568827,0.018291,1,274,5720,1436,6882
2,2,22,28091.912676,0.218163,1,191,7054,2154,9017
3,3,24,30877.905784,0.056401,1,309,7722,2621,10034
4,4,60,70623.587018,0.086901,1,607,5684,3317,8394
5,5,58,21453.342731,0.346435,0,24,0,0,-24
6,6,42,53572.777543,0.011187,1,733,9989,1425,10681
7,7,49,22237.342242,0.018677,1,137,6903,2074,8840
8,8,24,31033.415136,0.183788,1,405,10772,1663,12030
9,9,24,20692.050933,0.157120,1,535,7278,1542,8285


### 5. Проверка на однородность, применение критерия. Принятие решения. Расписать подробно с аргументами.

In [ ]:
imp_crit = 0.05
t, p_value = sp.stats.ttest_ind(test['NPV'], control['NPV'], alternative='greater', equal_var=False)
print('p-value:', p_value)
if (p_value <= imp_crit):
  print("Новый алгоритм дал значимое улучшение")
  test_bankruptcy = test[test['utilization'] == 1]['bankruptcy'].mean()
  control_bankruptcy = control[control['utilization'] == 1]['bankruptcy'].mean()
  print('Средняя вероятность банкротства среди активных клиентов в тесте:', test_bankruptcy)
  print('Средняя вероятность банкротства среди активных клиентов в контроле:', control_bankruptcy)

  if (test_bankruptcy >= control_bankruptcy):
    print("Риск банкротства среди активных клиентов увеличился, новый алгоритм необходимо доработать")
  else:
    print("Контрольная метрика не ухудшилась, новый алгоритм можно запускать")
else:
  print("Новый алгоритм не дал значимого улучшения")

p-value: 0.35553308578870113
Новый алгоритм не дал значимого улучшения


Незначимые результаты